In [1]:
import pandas as pd
import time
import numpy as np
from sklearn.cross_validation import train_test_split
import lightgbm as lgb
import gc
import matplotlib.pyplot as plt
import os
%matplotlib inline
from sklearn.linear_model import LogisticRegression
from scipy.special import expit, logit
from sklearn.metrics import roc_auc_score
from datetime import datetime as dt

almost_zero = 1e-10
almost_one = 1 - almost_zero


traininfo = [['./sub_dart_train_BayesOpt_2018_0501_0125_11.csv', './sub_dart_train_BayesOpt_2018_0501_0315_38.csv',
              './sub_dart_train_BayesOpt_2018_0501_0518_48.csv', './sub_dart_train_BayesOpt_2018_0501_0715_06.csv'],
             ['./sub_xgdt_train_BayesOpt_2018_0430_1519_33.csv', './sub_xgdt_train_BayesOpt_2018_0430_1656_17.csv', 
              './sub_xgdt_train_BayesOpt_2018_0430_1844_31.csv', './sub_xgdt_train_BayesOpt_2018_0430_2026_42.csv']
            ]

validinfo = [['./sub_dart_valid_BayesOpt_2018_0501_0125_11.csv', './sub_dart_valid_BayesOpt_2018_0501_0315_38.csv',
              './sub_dart_valid_BayesOpt_2018_0501_0518_48.csv', './sub_dart_valid_BayesOpt_2018_0501_0715_06.csv'],
             ['./sub_xgdt_valid_BayesOpt_2018_0430_1519_33.csv', './sub_xgdt_valid_BayesOpt_2018_0430_1656_17.csv', 
              './sub_xgdt_valid_BayesOpt_2018_0430_1844_31.csv', './sub_xgdt_valid_BayesOpt_2018_0430_2026_42.csv']
            ]

testinfo = [['./sub_dart_BayesOpt_2018_0501_0148_24.csv', './sub_dart_BayesOpt_2018_0501_0339_24.csv',
             './sub_dart_BayesOpt_2018_0501_0544_49.csv', './sub_dart_BayesOpt_2018_0501_0739_43.csv'],
            ['./sub_it_BayesOpt_2018_0430_1527_02.csv', './sub_it_BayesOpt_2018_0430_1703_36.csv',
             './sub_it_BayesOpt_2018_0430_1854_57.csv', './sub_it_BayesOpt_2018_0430_2035_07.csv']]

train_trueinfo = [['./rawdata_train_From2400_2018_0501_0125_11.csv', './rawdata_train_From2400_2018_0501_0315_38.csv',
                   './rawdata_train_From2400_2018_0501_0518_48.csv', './rawdata_train_From2400_2018_0501_0715_06.csv'],
                  ['./rawdata_valid_From2400_2018_0501_0125_11.csv', './rawdata_valid_From2400_2018_0501_0315_38.csv',
                   './rawdata_valid_From2400_2018_0501_0518_48.csv', './rawdata_valid_From2400_2018_0501_0715_06.csv']
                 ]
dtypes = {'is_attributed' : 'uint8'}

validlen = 5000000
models = []
starttime = time.time()
for i in range(4):
    print('Start fitting ' + str(i) + '-th model')
    data = pd.DataFrame()
    
    # dart
    traint = pd.read_csv(traininfo[0][i], usecols=['is_attributed'])
    valid = pd.read_csv(validinfo[0][i], usecols=['is_attributed'])
    train = traint.append(valid)
    del valid, traint; gc.collect()
    data['is_attributed_dart_1'] = train['is_attributed']
    del train; gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Read dart finished')

    # xgdt
    traint = pd.read_csv(traininfo[1][i], usecols=['is_attributed'])
    valid = pd.read_csv(validinfo[1][i], usecols=['is_attributed'])
    train = traint.append(valid)
    del valid, traint; gc.collect()
    data['is_attributed_xgdt_1'] = train['is_attributed']
    del train; gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Read xgdt finished')    

    data = data.clip(almost_zero, almost_one).apply(logit)
    dvalid = data[:validlen]
    data = data[validlen:]
    gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Split train finished')

    # y
    yt = pd.read_csv(train_trueinfo[0][i], usecols=['is_attributed'], dtype=dtypes)
    yvalid = pd.read_csv(train_trueinfo[1][i], usecols=['is_attributed'], dtype=dtypes)
    y = yt.append(yvalid)
    del yvalid, yt; gc.collect()
    yvalid = y[:validlen]
    y = y[validlen:]
    gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Read and Split y finished')

    # pred
    clf = LogisticRegression()
    clf.fit(data, y.values.ravel())
    print(f'[{time.time() - starttime: .2f}]: Fit finished')
    print('RocAuc Score: ' + str(roc_auc_score(yvalid.values.ravel(), clf.predict_proba(dvalid)[:, 1])))    
    
    models.append(clf)
    del data, dvalid, y, yvalid
    gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Model saved')

preds = pd.DataFrame()
for i in range(4):
    print('Start submission ' + str(i) + '-th model')
    test = pd.DataFrame()
    
    # dart
    train = pd.read_csv(testinfo[0][i], usecols=['is_attributed'])
    test['is_attributed_dart_1'] = train['is_attributed']
    del train; gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Read dart-test finished')

    # xgdt
    train = pd.read_csv(traininfo[1][i], usecols=['is_attributed'])
    test['is_attributed_xgdt_1'] = train['is_attributed']
    del train; gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Read xgdt-test finished')    

    test = test.clip(almost_zero, almost_one).apply(logit)
    gc.collect()
    preds['is_attributed_' + str(i)] = models[i].predict_proba(test)[:, 1]
    del test; gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Predict finished')
    sub = pd.read_csv('../input/test.csv', usecols=['click_id'])
    sub['is_attributed'] = preds['is_attributed_' + str(i)]
    datetime = dt.now().strftime('_%Y_%m%d_%H%M_%S')
    fname = '../sub/sub_Logistic_DartXGDT_Simple' + datetime + '.csv'
    sub.to_csv(fname, index=False)
    del sub; gc.collect()
    print(f'[{time.time() - starttime: .2f}]: Submission finished')
    
print(f'[{time.time() - starttime: .2f}]: Start making submission file')
sub = pd.read_csv('../input/test.csv', usecols=['click_id'])
coef = [0.25, 0.25, 0.25, 0.25]
sub['is_attributed'] = np.exp(coef[0]*np.log(preds['is_attributed_0'])
                              + coef[1]*np.log(preds['is_attributed_1'])
                              + coef[2]*np.log(preds['is_attributed_2'])
                              + coef[3]*np.log(preds['is_attributed_3']))

datetime = dt.now().strftime('_%Y_%m%d_%H%M_%S')
fname = '../sub/sub_Logistic_DartXGDT_' + datetime + '.csv'
sub.to_csv(fname, index=False)

del sub, preds; gc.collect()
print(f'[{time.time() - starttime: .2f}]: Subs created')

C:\Users\ahega\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Start fitting 0-th model
[ 13.59]: Read dart finished
[ 24.52]: Read xgdt finished
[ 26.94]: Split train finished
[ 77.41]: Read and Split y finished
[ 129.41]: Fit finished
RocAuc Score: 0.981633706814
[ 131.35]: Model saved
Start fitting 1-th model
[ 144.59]: Read dart finished
[ 155.44]: Read xgdt finished
[ 158.06]: Split train finished
[ 208.18]: Read and Split y finished
[ 256.17]: Fit finished
RocAuc Score: 0.986833114367
[ 258.22]: Model saved
Start fitting 2-th model
[ 271.42]: Read dart finished
[ 282.22]: Read xgdt finished
[ 284.74]: Split train finished
[ 335.17]: Read and Split y finished
[ 386.40]: Fit finished
RocAuc Score: 0.988202278171
[ 388.31]: Model saved
Start fitting 3-th model
[ 401.48]: Read dart finished
[ 412.18]: Read xgdt finished
[ 414.78]: Split train finished
[ 464.71]: Read and Split y finished
[ 513.56]: Fit finished
RocAuc Score: 0.982779449559
[ 515.50]: Model saved
Start submission 0-th model
[ 522.09]: Read dart-test finished
[ 534.47]: Read xgdt-